# Extraer un partido de Sofascore con `match_id` (ScraperFC)

Este cuaderno toma un `match_id` de Sofascore y utiliza **ScraperFC** para obtener:

- Información general del partido (equipos, marcador, fecha, estadio, etc.).
- Estadísticas del equipo.
- Estadísticas de jugadores.
- Posiciones promedio.
- Shotmap y momentum.

> Ajusta el `match_id` en la siguiente celda antes de ejecutar.


In [1]:
import pandas as pd
import ScraperFC as sfc
from pathlib import Path


## Configuración

Coloca tu `match_id` aquí. Puedes pegar directamente el ID o un `match_url` completo.


In [2]:
match_input = "15362090"  # <- Reemplaza con tu match_id o URL de Sofascore


In [3]:
def normalize_match_id(match_input: str) -> str:
    match_input = str(match_input).strip()
    if "#id:" in match_input:
        return match_input.split("#id:")[-1]
    return match_input

match_id = normalize_match_id(match_input)
match_id


'15362090'

## Inicializar ScraperFC


In [4]:
sofascore = sfc.Sofascore()


## Información general del partido


In [5]:
match_dict = sofascore.get_match_dict(match_id)

summary = {
    "match_id": match_id,
    "home_team": match_dict.get("homeTeam", {}).get("name"),
    "away_team": match_dict.get("awayTeam", {}).get("name"),
    "home_score": (match_dict.get("homeScore") or {}).get("display"),
    "away_score": (match_dict.get("awayScore") or {}).get("display"),
    "tournament": (match_dict.get("tournament") or {}).get("name"),
    "season": (match_dict.get("season") or {}).get("name"),
    "round": (match_dict.get("roundInfo") or {}).get("round"),
    "start_time": pd.to_datetime(match_dict.get("startTimestamp"), unit="s", errors="coerce"),
    "venue": (match_dict.get("venue") or {}).get("name"),
    "referee": (match_dict.get("referee") or {}).get("name"),
}

pd.DataFrame([summary])


Running


,match_id,home_team,away_team,home_score,away_score,tournament,season,round,start_time,venue,referee
0,15362090,Deportivo Garcilaso,Club Sporting Cristal,1,1,"Liga 1, Apertura",Liga 1 2026,1,2026-02-01 18:00:00,Estadio Inca Garcilaso de la Vega,Kevin Ortega


## ETL para Admin (Fantasy)

Este bloque transforma las estadísticas de ScraperFC al formato que espera el admin del Fantasy:
`player_id, match_id, goals, assists, minutesplayed, saves, fouls, yellow_cards, red_cards, clean_sheet, goals_conceded`.

- Usa columnas disponibles en `player_stats_df`.
- Completa `goals_conceded` a partir del marcador si no existe en el dataset.
- Calcula `clean_sheet` cuando corresponde (0/1) usando `goals_conceded` y minutos.


In [ ]:
# Extraer stats de jugadores desde SofaScore
try:
    player_stats_df = sofascore.scrape_player_match_stats(match_id)
except Exception as exc:
    print('No se pudo scrapear player stats:', exc)
    player_stats_df = pd.DataFrame()

if not isinstance(player_stats_df, pd.DataFrame):
    player_stats_df = pd.DataFrame(player_stats_df or [])

if player_stats_df.empty:
    print('player_stats_df vacio: revisa match_id o el scraper.')


In [6]:
import numpy as np

output_dir = Path("matches_details")
output_dir.mkdir(parents=True, exist_ok=True)


def coalesce_column(df: pd.DataFrame, candidates: list[str]) -> pd.Series:
    for col in candidates:
        if col in df.columns:
            return df[col]
    return pd.Series([None] * len(df))

def normalize_position(value: object) -> str:
    if value is None or pd.isna(value):
        return ""
    text = str(value).strip().lower()
    if text in {"gk", "goalkeeper", "goalie", "portero"}:
        return "GK"
    if text in {"defender", "def", "df", "d", "defensa"}:
        return "D"
    if text in {"midfielder", "mid", "mf", "m", "mediocampista"}:
        return "M"
    if text in {"forward", "fw", "f", "delantero"}:
        return "F"
    return text.upper()

player_id = coalesce_column(player_stats_df, ["id", "player_id", "player.id"])
team_id = coalesce_column(player_stats_df, ["teamId", "team_id", "team.id"])
position = coalesce_column(player_stats_df, ["pos", "playerPosition", "positionName", "player.position"]).apply(normalize_position)
minutesPlayed = coalesce_column(player_stats_df, ["minutesPlayed", "minutesplayed", "minutes"])
goals = coalesce_column(player_stats_df, ["goals"])
assists = coalesce_column(player_stats_df, ["assists"])
saves = coalesce_column(player_stats_df, ["saves", "savesMade"])
fouls = coalesce_column(player_stats_df, ["fouls", "foulsCommitted"])

home_team_id = match_dict.get("homeTeam", {}).get("id")
away_team_id = match_dict.get("awayTeam", {}).get("id")
home_score = (match_dict.get("homeScore") or {}).get("display")
away_score = (match_dict.get("awayScore") or {}).get("display")

def _to_int(value, default=0):
    try:
        if value is None or (isinstance(value, float) and np.isnan(value)):
            return default
        return int(float(value))
    except Exception:
        return default

team_goals_conceded = {}
if home_team_id is not None and away_team_id is not None:
    team_goals_conceded[home_team_id] = _to_int(away_score, None)
    team_goals_conceded[away_team_id] = _to_int(home_score, None)

clean_sheet = []
goals_conceded_filled = []

for idx in range(len(player_stats_df)):
    team_value = team_id.iloc[idx] if len(team_id) > idx else None
    minutes_val = minutesPlayed.iloc[idx] if len(minutesPlayed) > idx else 0
    position_val = position.iloc[idx] if len(position) > idx else ""
    is_defensive = position_val in {"GK", "D"}

yellow_cards = coalesce_column(player_stats_df, ["yellowCards", "yellow_cards", "yellow", "yc"])
red_cards = coalesce_column(player_stats_df, ["redCards", "red_cards", "red", "rc"])

admin_stats_df = pd.DataFrame({
    "player_id": player_id,
    "match_id": match_id,
    "minutesplayed": minutesPlayed,
    "goals": goals,
    "assists": assists,
    "saves": saves,
    "fouls": fouls,
    "yellow_cards": yellow_cards,
    "red_cards": red_cards,
    "clean_sheet": None,
    "goals_conceded": None,
})

admin_stats_df = admin_stats_df.fillna(0)
for col in ["player_id", "match_id", "minutesplayed", "goals", "assists", "saves", "fouls", "yellow_cards", "red_cards"]:
    admin_stats_df[col] = pd.to_numeric(admin_stats_df[col], errors="coerce").fillna(0).astype(int)

admin_stats_df.head()


NameError: name 'player_stats_df' is not defined

### Texto para pegar en Admin

Copia el siguiente bloque y pégalo en el textarea de Admin (cada fila es un jugador).


In [ ]:
admin_lines = admin_stats_df.apply(
    lambda row: ",".join(
        str(row[col])
        for col in [
            "player_id",
            "match_id",
            "minutesplayed",
            "goals",
            "assists",
            "saves",
            "fouls",
            "yellow_cards",
            "red_cards",
            "clean_sheet",
            "goals_conceded",
        ]
    ),
    axis=1,
)
admin_payload = "\n".join(admin_lines)
admin_payload[:1000]  # preview


'924850,15362082,0,0,90,2,0\n929014,15362082,0,0,90,0,0\n862216,15362082,0,0,90,0,0\n560054,15362082,0,0,90,0,3\n1159659,15362082,0,0,90,0,1\n789416,15362082,0,0,90,0,4\n777249,15362082,0,0,83,0,2\n975919,15362082,0,0,59,0,0\n1462908,15362082,0,0,89,0,0\n924574,15362082,0,0,59,0,0\n932933,15362082,0,0,59,0,3\n1107888,15362082,1,0,31,0,0\n846474,15362082,0,0,31,0,1\n340529,15362082,0,0,31,0,2\n1503648,15362082,0,0,13,0,1\n883293,15362082,0,0,1,0,0\n883309,15362082,0,0,0,0,0\n1017755,15362082,0,0,0,0,0\n805055,15362082,0,0,0,0,0\n989259,15362082,0,0,0,0,0\n883774,15362082,0,0,90,3,0\n1121487,15362082,0,0,90,0,0\n991108,15362082,0,0,90,0,0\n874980,15362082,0,0,90,0,1\n805366,15362082,0,0,90,0,2\n989228,15362082,0,0,89,0,0\n1170641,15362082,0,0,90,0,3\n976302,15362082,1,0,70,0,2\n1016974,15362082,0,0,90,0,3\n883299,15362082,0,0,83,0,1\n1098306,15362082,0,0,83,0,0\n883729,15362082,0,0,20,0,2\n1089808,15362082,0,0,13,0,0\n972824,15362082,0,0,13,0,0\n1022275,15362082,0,0,1,0,0\n2454293,153620

In [ ]:
admin_stats_df.to_csv(output_dir / f"Fantasy_{match_id}.csv", index=False)


### Vista rápida de los datos


In [ ]:
player_stats_df.head()


,name,slug,shortName,position,jerseyNumber,height,userCount,gender,id,country,...,progressiveBallCarriesCount,accurateCross,blockedScoringAttempt,bigChanceMissed,shotOffTarget,keyPass,goals,bigChanceCreated,teamName,captain
0,Diego Melián,diego-melian,D. Melián,G,22,189.0,49,M,924850,"{'alpha2': 'UY', 'alpha3': 'URY', 'name': 'Uru...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Boys,NaN
1,Oslimg Mora,oslimg-mora,O. Mora,D,24,172.0,61,M,929014,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Boys,NaN
2,Renzo Alfani,renzo-alfani,R. Alfani,D,4,186.0,16,M,862216,"{'alpha2': 'AR', 'alpha3': 'ARG', 'name': 'Arg...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Boys,NaN
3,Hansell Riojas,riojas-hansell,H. Riojas,D,26,183.0,33,M,560054,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sport Boys,True
4,Sebastian Aranda,sebastian-aranda,S. Aranda,D,18,180.0,35,M,1159659,"{'alpha2': 'PE', 'alpha3': 'PER', 'name': 'Per...",...,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,Sport Boys,NaN
